In [1]:
import numpy as np
import shutil
import os
import pickle
import matplotlib.pyplot as plt
import pandas as pd

from functions_py3 import *

In [20]:
# files = os.listdir()

# for file in files:
#     if file.startswith('grenland_bridge') or file.startswith('1-Skrastag') or file.startswith('2-Viadukt'):
#         os.remove(file)
#     elif file.endswith('.npy'):
#         os.remove(file)

In [21]:
steel_e_modulus = 2.1E+11
steel_density = 7850
concrete_e_modulus = 0.27E+11
concrete_density = 2500
joint_stiffness = 1

theta = np.array([steel_e_modulus, steel_density, concrete_e_modulus, concrete_density, joint_stiffness])

# Global parameter bounds
lower_allowable_theta = np.array([2.1E+10, 7000, 0.27E+10, 2000, 1])
upper_allowable_theta = np.array([2.1E+12, 9000, 0.27E+12, 3000, 1000])

# Local parameter bounds
lb_local = np.array([0.90, 0.90, 0.90, 0.90, 0.90])
ub_local = np.array([1.10, 1.10, 1.10, 1.10, 1.10])

# Local parameter bounds, unscaled
theta_min = lb_local*theta
theta_max = ub_local*theta

p = len(theta)

# Check for exceedance of global bounds
for index in range(p):

    if theta_min[index] <= lower_allowable_theta[index]:
        theta_min[index] = np.copy(lower_allowable_theta[index])

    if theta_max[index] >= upper_allowable_theta[index]:
        theta_max[index] = np.copy(upper_allowable_theta[index])

# Upper and lower parameter bounds
d_theta_min = theta_min - theta
d_theta_max = theta_max - theta

# Upper and lower parameter bounds, scaled
d_theta_min_s = d_theta_min / theta
d_theta_max_s = d_theta_max / theta


lb = np.copy(d_theta_min_s)
ub = np.copy(d_theta_max_s)

print(lb)
print(ub)

[-0.1 -0.1 -0.1 -0.1  0. ]
[0.1 0.1 0.1 0.1 0.1]


In [ ]:
parameter_list_pt = np.load("parameter_list_pt.npy")
parameter_list_upd = np.load("parameter_list_upd.npy")
parameter_list = np.load("parameter_list.npy")

print(parameter_list_pt)
print(parameter_list_upd)
print(parameter_list)

In [2]:
with open('info.pkl', 'rb') as info_file:
    info = pickle.load(info_file)

theta = info['theta']


In [30]:
lamda = info["lamda"]
lamda

lamda_m = info["lamda_m"]
itr = 2

print(((lamda_m - lamda[itr+1, :])/lamda[0, :])**2)
print()
print(np.sum(((lamda_m - lamda[itr+1, :])/lamda[0, :])**2))


print(f"\n({lamda_m[-1]} - {lamda[2, -1]} / {lamda[0, -1]})^2\n")

print(lamda[0,:])

[6.55113080e-03 1.06882635e-04 7.42092086e-03 1.00320550e-02
 3.24608748e-04 2.02324352e-01 2.01888570e-02 9.42233864e-03
 5.16578023e-02 2.32014636e-06 1.55112504e-02 2.11445165e-04
 2.56674460e-03 1.18122126e-02 2.71179607e-02 9.30988630e-03
 2.64858062e-02 1.15409224e-02 1.01409336e-03 4.91300148e-03
 5.68272886e-02 5.46512898e-01 2.92711825e-01 1.22526921e-02
 2.84648858e-01 2.52773717e-01 9.78239854e-01 6.78290273e-01
 3.13425383e-01 4.75920796e-01 2.27870489e-02 2.60493397e-01
 7.38213733e-01 3.30211191e-02]

5.3646334738503585

(1.0 - 0.84666 / 0.87273)^2

[0.28048122 0.38440788 0.69401562 0.93602175 1.36617315 0.84079778
 1.63210654 1.84099436 2.62996912 2.19965005 2.24076986 3.24923873
 2.72408247 3.51331115 4.24417067 3.38769436 5.73433208 0.90339
 0.96876    0.93205    0.77635    0.72963    0.65747    0.91172
 0.5915     0.66882    0.47898    0.56242    0.72597    0.616
 0.87073    0.67594    0.54195    0.87273   ]


In [32]:
lamda[15,0:17]

array([0.25688508, 0.35304701, 0.63656777, 0.87016588, 1.27214336,
       0.98126268, 1.5408529 , 1.79002178, 2.41004014, 2.05857706,
       2.18721175, 2.54599357, 2.54599357, 3.29628181, 3.93358159,
       3.23060894, 5.43317032])

In [92]:
freqs_num = np.load('03_Results/grenland_bridge_frequencies_all.npy')
modes_num = np.load('03_Results/grenland_bridge_modes_all.npy')

freqs_est = np.load('01_Initial/02_Measured/grenland_bridge_frequencies.npy')
modes_est = np.load('01_Initial/02_Measured/grenland_bridge_modes.npy')

(MAC_initial_model, MMI_initial_model, results_MAC, results_MMI) = get_MAC_MMI(freqs_est, modes_est,
                                                     freqs_num, modes_num,
                                                     filtering=False)



In [94]:
mode_list = []
iterations = 6



for i in range(3, 17):
    mode_list.append('Mode ' + str('{:02}'.format(i+1)))

q = len(mode_list)
lamda = np.zeros((iterations + 1, q), dtype=float)

for idx, mode in enumerate(mode_list):
    string_idx = mode[-2:]
    if string_idx[0] == 0:
        f_idx = int(mode[-1]) - 1
    else:
        f_idx = int(string_idx) - 1
    lamda[0, idx] = results_MAC[mode][0, 2]

print(lamda)

[[5.73433208 1.36617315 0.84079778 1.63210654 3.51331115 2.62996912
  2.31704783 2.24076986 3.24923873 3.63462591 3.51331115 1.63210654
  3.38769436 5.73433208]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.      

In [60]:
for idx, mode in enumerate(mode_list):
    string_idx = mode[-2:]
    if string_idx[0] == 0:
        f_idx = int(mode[-1]) - 1
    else:
        f_idx = int(string_idx) - 1
    lamda[0, idx] = results_MMI[mode][0, 2]

print(lamda)

[[0.93712276 1.37209737 1.29302835 1.68169856 1.89040256 2.63054585
  2.20330834 2.59171367 2.76301599 2.76301599 3.52046657 4.29072332
  3.69746256 5.93151903 6.05347967 6.61364698]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.   

In [133]:
lamda_m = np.zeros(len(mode_list)*2, dtype=float)
lamda_start = np.zeros(len(mode_list)*2, dtype=float)

W = np.zeros((len(lamda_m), len(lamda_m)))

for i, mode in enumerate(mode_list):

    if i < 4:
        W[i, i] = (2/3) / len(mode_list) + (1/40) * ((len(mode_list)-4)/len(mode_list))
        W[i + len(mode_list), i + len(mode_list)] = (1/3) / len(mode_list) + (1/40) * ((len(mode_list)-4)/len(mode_list))
    else:
        W[i, i] = (2/3) / len(mode_list) - (1/40) * (4/len(mode_list))
        W[i + len(mode_list), i + len(mode_list)] = (1/3) / len(mode_list) - (1/40) * (4/len(mode_list))

    lamda_start[i] = results_MMI[mode][0, 2]
    lamda_start[i + len(mode_list)] = 1


# np.sum(np.diag(W))
np.diag(W)

array([0.06547619, 0.06547619, 0.06547619, 0.06547619, 0.04047619,
       0.04047619, 0.04047619, 0.04047619, 0.04047619, 0.04047619,
       0.04047619, 0.04047619, 0.04047619, 0.04047619, 0.04166667,
       0.04166667, 0.04166667, 0.04166667, 0.01666667, 0.01666667,
       0.01666667, 0.01666667, 0.01666667, 0.01666667, 0.01666667,
       0.01666667, 0.01666667, 0.01666667])

In [29]:
x = [0, 1, 2, 3, 4, 5]

test = [i for i in range(int(len(x)/2))]

test

[0, 1, 2]

In [34]:
G = info['G_matrices']['G5']

rows, cols = np.shape(G)

rows

[i for i in range(int(rows/2))]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

In [43]:
[f"$MAC_({i+4})$" for i in range(int(rows/2))]

['$MAC_(4)$',
 '$MAC_(5)$',
 '$MAC_(6)$',
 '$MAC_(7)$',
 '$MAC_(8)$',
 '$MAC_(9)$',
 '$MAC_(10)$',
 '$MAC_(11)$',
 '$MAC_(12)$',
 '$MAC_(13)$',
 '$MAC_(14)$',
 '$MAC_(15)$',
 '$MAC_(16)$',
 '$MAC_(17)$',
 '$MAC_(18)$',
 '$MAC_(19)$']

In [76]:
G = info["G_matrices"]["G5"]

G = G / np.amax(G)

for i in range(16):
    print(G[i + int(rows/2),:])

[-0.05501618  0.01941748 -0.00647249  0.04530744  0.        ]
[-0.11326861  0.04854369 -0.35275081  0.08737864 -0.05177994]
[-0.00970874  0.00970874  0.         -0.00323625  0.        ]
[ 0.01618123 -0.00970874  0.00323625 -0.02588997  0.        ]
[0.01294498 0.00323625 0.         0.01294498 0.00323625]
[ 0.04854369 -0.01294498  0.00323625 -0.02588997  0.        ]
[1.         0.24919094 0.37864078 0.49190939 0.37216828]
[-0.00970874  0.01294498  0.00323625  0.          0.        ]
[-0.09385113  0.03559871 -0.00970874  0.08414239  0.00323625]
[ 0.15210356 -0.02265372  0.02265372 -0.04854369  0.00323625]
[ 0.09708738 -0.00970874  0.00970874 -0.05177994 -0.00323625]
[ 0.04854369  0.03236246 -0.01941748 -0.03236246  0.        ]
[-0.01618123  0.00323625 -0.00323625 -0.00970874 -0.00647249]
[-0.05501618  0.01618123  0.00970874  0.02588997 -0.01941748]
[-0.00323625  0.          0.01618123 -0.00323625  0.        ]
[ 0.18122977 -0.07443366  0.05825243 -0.15210356 -0.00323625]


In [3]:
MAC_initial = info['MAC_initial']
MAC_updated = info['MAC_updated']['MAC_updated50']

MMI_initial = info['MMI_initial']
MMI_updated = info['MMI_updated']['MMI_updated50']

ind_list_initial = []
ind_list_updated = []
naming_rows_initial = []
naming_rows_updated = []

for i in range(len(MAC_initial[1])):
    ind = np.argsort(MMI_initial[:,i])[::-1]
    ind_list_initial.append(ind[0])
    naming_rows_initial.append(ind[0] + 1)

MAC_initial_sorted = MAC_initial[ind_list_initial,:]
MAC_initial_sorted_diag = np.diag(MAC_initial_sorted)

for i in range(len(MAC_updated[1])):
    ind = np.argsort(MMI_updated[:,i])[::-1]
    ind_list_updated.append(ind[0])
    naming_rows_updated.append(ind[0] + 1)

MAC_updated_sorted = MAC_updated[ind_list_updated,:]
MAC_updated_sorted_diag = np.diag(MAC_updated_sorted)

MAC_change = MAC_updated_sorted_diag - MAC_initial_sorted_diag

ind_list_initial = []
ind_list_updated = []
naming_rows_initial = []
naming_rows_updated = []

for i in range(len(MMI_initial[1])):
    ind = np.argsort(MMI_initial[:,i])[::-1]
    ind_list_initial.append(ind[0])
    naming_rows_initial.append(ind[0] + 1)

MMI_initial_sorted = MMI_initial[ind_list_initial,:]
MMI_initial_sorted_diag = np.diag(MMI_initial_sorted)

for i in range(len(MMI_updated[1])):
    ind = np.argsort(MMI_updated[:,i])[::-1]
    ind_list_updated.append(ind[0])
    naming_rows_updated.append(ind[0] + 1)

MMI_updated_sorted = MMI_updated[ind_list_updated,:]
MMI_updated_sorted_diag = np.diag(MMI_updated_sorted)

MMI_change = MMI_updated_sorted_diag - MMI_initial_sorted_diag

measured_fn = info['lamda_m'][0:len(MAC_change)]
mode_array = np.array([i+1 for i in range(0, len(measured_fn))])

initial_fn = info['lamda'][0, 0:len(MAC_change)]
initial_error = np.abs(measured_fn - initial_fn) / measured_fn * 100

updated_fn = info['lamda'][50, 0:len(MAC_change)]
updated_error = np.abs(measured_fn - updated_fn) / measured_fn * 100


column_names = ["Mode", "Measured f_n", "Initial f_n", "Error", "Updated f_n", "Error", 
                "Initial MAC", "Updated MAC", "Change", "Initial MMI", "Updated MMI", "Change"]

columns = np.array([tuple(mode_array), tuple(np.round(measured_fn, 4)), tuple(np.round(initial_fn, 4)), tuple(np.round(initial_error, 2)),
                    tuple(np.round(updated_fn, 4)), tuple(np.round(updated_error, 2)), tuple(MAC_initial_sorted_diag), 
                    tuple(MAC_updated_sorted_diag), tuple(MAC_change),
                    tuple(MMI_initial_sorted_diag), tuple(MMI_updated_sorted_diag), tuple(MMI_change)])

mac_table = pd.DataFrame(columns.T, columns=column_names)

mac_table

,Mode,Measured f_n,Initial f_n,Error,Updated f_n,Error,Initial MAC,Updated MAC,Change,Initial MMI,Updated MMI,Change
0,1.0,0.2999,0.2853,4.87,0.2689,10.35,0.89870,0.89926,0.00056,0.424981,0.397901,-0.027080
1,2.0,0.3839,0.3965,3.27,0.3579,6.79,0.96586,0.96940,0.00354,0.466562,0.450743,-0.015819
2,3.0,0.6197,0.7004,13.03,0.6382,2.99,0.92777,0.93941,0.01164,0.398725,0.454731,0.056005
3,4.0,0.8202,0.9371,14.25,0.8653,5.50,0.77412,0.85338,0.07926,0.315826,0.399209,0.083383
4,5.0,1.0876,1.3722,26.17,1.0746,1.19,0.74177,0.53362,-0.20815,0.240014,0.260851,0.020837
5,6.0,1.2205,1.2991,6.44,1.1893,2.55,0.67516,0.68111,0.00595,0.305359,0.327781,0.022422
6,7.0,1.3607,1.6837,23.74,1.5141,11.27,0.88602,0.87885,-0.00717,0.324315,0.383054,0.058739
7,8.0,1.6175,1.8918,16.96,1.7470,8.01,0.76046,0.83353,0.07307,0.295440,0.376720,0.081280
8,9.0,1.9312,2.6306,36.21,2.3747,22.97,0.66716,0.65681,-0.01035,0.152510,0.213579,0.061069
9,10.0,2.1114,2.2030,4.34,1.9928,5.62,0.47433,0.55864,0.08431,0.215454,0.251242,0.035789


In [4]:
np.sum(MMI_change)

0.6157874734967104

In [4]:
parameter = [r'$E_s$', r'$g_s$', r'$E_c$', r'$g_c$', r'$k_y$', r'$k_z$', 
             r'$k_{\phi_x}$', r'$k_{\phi_y}$', r'$k_{\phi_z}$']

parameter_ref_value = info["theta"][0]

parameter_upd_value = info["theta"][50]

parameter_change = (parameter_upd_value - parameter_ref_value) / parameter_ref_value * 100

parameter_column = ["Parameter", "Referance value", "Updated value", "Change"]

parameter_columns = np.array([parameter, (parameter_ref_value), parameter_upd_value, np.around(parameter_change, 2)])

parameter_table = pd.DataFrame(parameter_columns.T, columns=parameter_column)

parameter_table

,Parameter,Referance value,Updated value,Change
0,$E_s$,210000000000.0,179580354666.5698,-14.49
1,$g_s$,7850.0,9000.0,14.65
2,$E_c$,27000000000.0,33964375118.447334,25.79
3,$g_c$,2500.0,3000.0,20.0
4,$k_y$,10000000000.0,42998169595.81869,329.98
5,$k_z$,10000000000.0,318733999.10989374,-96.81
6,$k_{\phi_x}$,1000000000000.0,5706307519597.273,470.63
7,$k_{\phi_y}$,1000000000000.0,3144763407.537547,-99.69
8,$k_{\phi_z}$,1000000000000.0,3439853567544.66,243.99


In [11]:
print(info["theta"][50])

[1.79580355e+11 9.00000000e+03 3.39643751e+10 3.00000000e+03
 4.29981696e+10 3.18733999e+08 5.70630752e+12 3.14476341e+09
 3.43985357e+12]


In [17]:
G = np.abs(info['G_matrices']['G50'])

G[17:,:].shape

(17, 9)